# TOC:
[I. Evaluate Machine Learning Algorithms](#I.-Evaluate-Machine-Learning-Algorithms)<br>
[1.1 Split into Train and Test Sets (hold out)](#1.1-Split-into-Train-and-Test-Sets-(hold-out))<br>
[1.2 K-fold Cross Validation](#1.2-K-fold-Cross-Validation)<br>
[1.3 Leave One Out Cross Validation](#1.3-Leave-One-Out-Cross-Validation)<br>
[1.4 Repeated Random Test-Train Splits](#1.4-Repeated-Random-Test-Train-Splits)<br>

[II. Conclusions](#II.-Conclusions)

## I. Evaluate Machine Learning Algorithms

Không thể train và test trên cùng một dữ liệu bởi điều đó sẽ gây ra overfitting. Nên chúng ta phải đánh giá thuật toán trên dữ liệu không dùng để train.<br>
Đánh giá ở đây là cách đo lường khi chúng ta nghĩ về việc, thuật toán sẽ hoạt động như thế nào trong thực tiễn.

In [1]:
import pandas as pd
import numpy as np
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv('pima-indians-diabetes.csv', names=names)
array = dataframe.values
X = array[:,0:8] # input
Y = array[:,8] # output
seed = 7 # seed cho random state
dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### 1.1 Split into Train and Test Sets (hold out)

Phương thức đơn giản nhất để đánh giá sự biểu diễn của thuật toán trong machine learning là sử dụng tập train và tập test khác nhau. Chúng ta có thể tách bộ dữ liệu gốc thành 2 phần train và test.<br>
Train thuật toán trên phần đầu và predict trên phần sau.<br>
Kĩ thuật đánh giá này rất nhanh. Nó lý tưởng trên bộ dữ liệu lớn (hàng triệu bảng ghi) khi đó có bằng chứng mạnh mẽ rằng việc tách dữ liệu đại diện cho vấn đề đang quan tâm.<br> 
Và cũng bởi vì tốc độ nên nó hữu ích cho việc tiếp cận khi thuật toán bạn đang đầu tư vào chậm khi train. Mặt trái là, nó có thể có variance lớn. Và sự khác nhau giữa tập train và test thì đưa đến hệ quả là sự khác biệt về ý nghĩa trong mức độ đo lường độ chính xác.<br>
Ví dụ bên dưới chia train và test thành 67%, 33%, dùng mô hình Logistic Regression.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)
model = LogisticRegression()
model.fit(X_train, Y_train)
result = model.score(X_test, Y_test)
print('Accuracy: %.3f%%' % (result*100))

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy: 75.591%


Chúng ta có thể thấy đánh giá độ chính xác của mô hình xấp xỉ 75%.<br>
**Chú ý** rằng chúng ta có chung kết quả này vì chúng ta đã cụ thể kích cỡ tách (test = 0.33) và random seed (seed =7)<br>
Điều này là **quan trọng** nếu chúng ta muốn so sánh kết quả đo lường độ chính xác với một thuật toán khác hoặc cùng thuật toán nhưng với cấu hình khác (configuration). Để đảm bảo sự so sánh này là công bằng, ta phải chắc rằng chúng ta đã train và test trên cùng một dữ liệu.

### 1.2 K-fold Cross Validation

Cross Validation là một cách tiếp cận dùng để đo lường biểu diễn của một thuật toán machine learning với ít variance hơn là một train test split riêng lẻ. Nó hoạt động bằng việc tách bộ dữ liệu thành k phần (ví dụ k = 3, k=5). Mỗi phần tách được gọi là 1 fold. Thuật toán được train trên k folds, với một phần giữ lại và được test trên phần dữ lại của fold đó.<br>
Việc này được lặp lại nên mỗi fold trên bộ dữ liệu có cơ hội là một phần giữ lại của bộ test. Sau khi chạy cross validation, sẽ có k số điểm khác nhau, bạn có thể tổng kết lại bằng mean hoặc std.<br>
K được chọn phải ứng với kích cỡ của phần test, và phải đủ lớn để trở thành một sample có cơ sở cho vấn đề đặt ra.<br>
Cho những bộ dữ liệu có kích thước khiêm tốn, giá trị k thường là 3, 5 hoặc 10. Ví dụ bên dưới là k =10.

In [16]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
num_folds = 10
seed = 7
model = LogisticRegression(solver='liblinear') # solver có thể thay đổi nếu để mặc định
kfold = KFold(n_splits= num_folds, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 76.951% (4.841%)


Độ chính xác xấp xỉ 77% và dao động trong khoảng 4.8%

### 1.3 Leave One Out Cross Validation

In [4]:
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.linear_model import LogisticRegression
loocv = LeaveOneOut()
model = LogisticRegression(solver='liblinear')
results = cross_val_score(model, X, Y, cv=loocv)
print('Accuracy %.3f%% (%.3f%%)' % (results.mean()*100, results.std()*100))

Accuracy 76.823% (42.196%)


Điểm số của std ở phương pháp này có variance lớn hơn hẳn so với k-fold cross validation ở trên.

### 1.4 Repeated Random Test-Train Splits

Một biến thể khác của k-fold cross validation là tạo ra một phần tách ngẫu nhiên của dữ liệu, nhưng lặp lại quá trình tách và đánh giá thuật toán này nhiều lần như cross validation.<br> 
Nó có tốc độ như sử dụng train test split, và giảm được variance khi đo lường biểu diễn trong k-fold cross validation. Bạn có thể lặp lại nhiều lần nếu cần để cải thiện độ chính xác.<br>
Mặt yếu của kĩ thuật này là những cái lặp lại này có thể trùng với trùng với dữ liệu bộ train và test trước đó dẫn đến việc đánh giá bị thừa.<br>
Ví dụ bên dưới tách dữ liệu train test thành 67% và 33% và lặp lại quá trình này 10 lần.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit, cross_val_score
model = LogisticRegression(solver='liblinear')
seed = 7
test_size = 0.33
n_splits = 10
kfold = ShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print('Accuracy %.3f%% (%.3f%%)' % (results.mean()*100, results.std()*100))

Accuracy 76.535% (1.691%)


Chúng ta có thể thấy rằng ở lần này, thì điểm số gần với nhau hơn (std thấp)

## II. Conclusions

### Những kĩ thuật này được dùng khi nào?
- Nhìn chung k-fold cross validation là một kĩ thuật chuẩn để đánh giá biểu diễn của mô hình cho dữ liệu chưa từng được nhìn thấy. Với k là 3,5 hoặc 10.
- Sử dụng train test split (hold out) thì tốc độ nhanh và thích hợp với các thuật toán training chậm hoặc bộ dữ liệu **lớn** với bias thấp hơn.
- Kĩ thuật như leave one out và repeated random test train thì hữu ích ở tầm trung khi cố gắng cân bằng variance trong việc đo lường biểu diễn, tốc độ training mô hình và và kíck cỡ dữ liệu.<br>

Cách tốt nhất là trải nghiệm và tìm ra kĩ thuật cho vấn đề của bạn là nhanh và đưa ra được những đo lường lý tưởng cho sự biểu diễn. Nếu còn hồ nghi thì dùng 10-fold cross validation.